# Import Libraries

In [1]:
# Import necessary libraries to get started
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from tqdm import tqdm
import pickle
import os

# Data Preprocessing

In [2]:
# Import the dataset
def get_data():
    """
    Load SEED data from .npz file.
    
    :return: Combined train data, combined test data, train labels, test labels.
    """
    feature_1s_dir = '../EEG-DE-feature/eeg_used_1s'
    file_1s_list = os.listdir(feature_1s_dir)
    file_1s_list.sort()
    
    all_train_data = {key: [] for key in ['delta', 'theta', 'alpha', 'beta', 'gamma']}
    all_test_data = {key: [] for key in ['delta', 'theta', 'alpha', 'beta', 'gamma']}
    all_train_labels = []
    all_test_labels = []
    
    for item in tqdm(file_1s_list, desc="Loading data"):
        npz_data = np.load(os.path.join(feature_1s_dir, item))
        
        # Extract train and test data
        train_data = pickle.loads(npz_data['train_data'])
        test_data = pickle.loads(npz_data['test_data'])
        
        # Append data from this file to the overall dictionary
        for key in all_train_data.keys():
            all_train_data[key].append(train_data[key])
            all_test_data[key].append(test_data[key])
        
        all_train_labels.append(npz_data['train_label'])
        all_test_labels.append(npz_data['test_label'])
        
    # Combine train and test data across all files
    train_data = {key: np.concatenate(all_train_data[key], axis=0) for key in all_train_data.keys()}
    test_data = {key: np.concatenate(all_test_data[key], axis=0) for key in all_test_data.keys()}
    train_labels = np.concatenate(all_train_labels, axis=0)
    test_labels = np.concatenate(all_test_labels, axis=0)
    
    # Convert labels: -1 (negative) -> 0, 0 (neutral) -> 1, 1 (positive) -> 2
    # train_labels = train_labels + 1
    # test_labels = test_labels + 1
    
    # Combine features from all bands into a single feature vector
    trainX = np.hstack([train_data[key] for key in train_data.keys()])
    testX = np.hstack([test_data[key] for key in test_data.keys()])
    
    # Debug information
    print("Combined Train Data Shape:", trainX.shape)
    print("Combined Test Data Shape:", testX.shape)
    print("Combined Train Labels Shape:", train_labels.shape)
    print("Combined Test Labels Shape:", test_labels.shape)
    
    return trainX, testX, train_labels, test_labels

# Modelling, Training and Test

In [3]:
#import sklearn's model selection and split the data set into %80 training and %20 test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = get_data()

Loading data: 100%|██████████| 36/36 [00:00<00:00, 42.77it/s]


Combined Train Data Shape: (72360, 310)
Combined Test Data Shape: (49824, 310)
Combined Train Labels Shape: (72360,)
Combined Test Labels Shape: (49824,)


In [4]:
#import KNeighborsClassifier and KNN on the training dataset
from sklearn.neighbors import KNeighborsClassifier
estimator = KNeighborsClassifier(n_neighbors=3)
estimator.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [5]:
#apply the trained estimator on the test dataset
mean_accuracy=estimator.score(X_test, y_test)
y_pred = estimator.predict(X_test)
print(mean_accuracy)

d:\BCI\Final_project\.venv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 系統找不到指定的檔案。
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\BCI\Final_project\.venv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Program Files\Python310\lib\subprocess.py", line 501, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Program Files\Python310\lib\subprocess.py", line 969, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\Python310\lib\subprocess.py", line 1438, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


0.6903299614643545


In [6]:
# output the predicted values
y_pred

array([2., 2., 2., ..., 2., 2., 2.])

In [7]:
# import relevant metrics and print the confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 7340  5550  2914]
 [ 2809 12119  1992]
 [ 1151  1013 14936]]
              precision    recall  f1-score   support

         0.0       0.65      0.46      0.54     15804
         1.0       0.65      0.72      0.68     16920
         2.0       0.75      0.87      0.81     17100

    accuracy                           0.69     49824
   macro avg       0.68      0.68      0.68     49824
weighted avg       0.68      0.69      0.68     49824

